In [1]:
!python -V

Python 3.9.19


In [2]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

Bad pipe message: %s [b'0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nHost: localhost:34669\r\nUs', b'-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.']
Bad pipe message: %s [b'0.0 Safari/537.36 Edg/125.0.0.0\r\nAccept-Encodin']
Bad pipe message: %s [b' gzip, deflate, br, zstd\r\nAccept-Language: en-US,en;q=0.9,en-CA;q=0.8\r\nCache-Control: max-age=0\r\nRefer']
Bad pipe message: %s [b': https://symmetrical-orbit-4vq6v55vgv5h7g6p.github.dev/\r\nX-Request-ID: d038899a92289b064bf4f6540280']
Bad pipe message: %s [b'9c\r\nX-Real-IP: 10.240.3.217\r\nX-Forwarded-Port: 44', b'\nX-Forwarded-Scheme: https\r\nX-Original-URI: /\r\nX-S', b'eme: https\r\nsec-fetch-site: same-site\r\nsec-fetch-mode: navigate\r\nsec-fetch-dest: document\r\nsec-ch-']
Bad pipe message: %s [b': "Microsoft Edge";v="125", "Chromium";v="125", "Not.A/Brand";v="24"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Win', b'ws"\r\nprior

In [5]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("new-taxi-experiment")

<Experiment: artifact_location='/workspaces/experiment-tracking-zoomcamp/02-experiment-tracking/mlruns/4', creation_time=1716259251562, experiment_id='4', last_update_time=1716259251562, lifecycle_stage='active', name='new-taxi-experiment', tags={}>

In [6]:
import mlflow

# Set the tracking URI to the SQLite database
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Set the experiment name
mlflow.set_experiment("new-taxi-experiment")

# Start a new run
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 0.85)
    with open("output.txt", "w") as f:
        f.write("Hello, world!")
    mlflow.log_artifact("output.txt")

print("Run logged to experiment 'new-taxi-experiment'.")


Run logged to experiment 'new-taxi-experiment'.


In [6]:
import mlflow

# Set the tracking URI to the SQLite database
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Set the experiment name
mlflow.set_experiment("new-taxi-experiment")

print("Experiment setup complete")


Experiment setup complete


In [7]:
import mlflow

# Set the tracking URI to the SQLite database
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Set the experiment name
experiment = mlflow.set_experiment("new-taxi-experiment")
print(f"Experiment details: {experiment}")

# Start a new run
with mlflow.start_run() as run:
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 0.85)
    with open("output.txt", "w") as f:
        f.write("Hello, world!")
    mlflow.log_artifact("output.txt")
    print(f"Run ID: {run.info.run_id}")

print("Run logged to experiment 'new-taxi-experiment'.")


Experiment details: <Experiment: artifact_location='/workspaces/experiment-tracking-zoomcamp/02-experiment-tracking/mlruns/4', creation_time=1716259251562, experiment_id='4', last_update_time=1716259251562, lifecycle_stage='active', name='new-taxi-experiment', tags={}>
Run ID: 6236d7cac2fd4f1ea76b19f4029c3d36
Run logged to experiment 'new-taxi-experiment'.
